In [14]:
import os

###################################################################################
# Check that these are the correct paths running (change if necessary)

# Finn's Laptop
studydir = '/mnt/d/KPUM_NODDI/Data'
codedir = os.path.join('/home/finn/Code/KPUM_NODDI_ssd','kpum_noddi')
codedir = os.path.join('/home/finn/Code/','kpum_noddi')

# KPUM Workstation
studydir = '/mnt/e/Finn/KPUM_NODDI/Data'
codedir = os.path.join('/home/radio/KPUM_NODDI','code/kpum_noddi')

# Parallell diffusivity parameter for NODDI-model for NODDI estimation
dPar = 1.7e-3

###################################################################################
# USER INPUT - START

# Participant details
subject = '' # e.g. 035
session = '' # e.g. MR1
derivatives = '' # e.g. derivatives/dMRI_np' or 'derivatives/dMRI_op'
protocol = '' # 'NEW' or 'ORIG'

# USER INPUT - END
##################################################################################

# tsv-files to keep track of in studydir/niftibasefolder
subjecttrackerfile = 'Subject_Tracker_for_dmri_dtidkinoddi_pipeline.tsv'


In [ ]:
import os

# Define I/O folders and files
#niftifolder = os.path.join(studydir, niftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
datadir = os.path.join(studydir, derivatives, os.path.join(f'sub-{subject}',f'ses-{session}'))
if not os.path.exists(datadir): # then make this directory
    os.makedirs(datadir)
# TSV-file subject tracker file
subjecttrackertsv = os.path.join(studydir, derivatives, subjecttrackerfile)

def perform_process(processcall) :
    import os, subprocess
    # Perform the process given by the processcall by launching into terminal
    p=subprocess.Popen(processcall, stdout=subprocess.PIPE, shell=True)
    # Poll process.stdout to show stdout live
    while True:
        output = p.stdout.readline()
        if p.poll() is not None:
            break
        if output:
            print(output.strip().decode("utf-8"))
    rc = p.poll()


########################################
## START

import os 
import pandas as pd

currdir = os.getcwd()
os.chdir(studydir)

## Process to perform - dmri_dtidkinoddi_pipeline
process = 'dmri_dtidkinoddi_pipeline'
processfile = 'dmri_dtidkinoddi_pipeline.sh'
processfilepath = os.path.join(codedir,'shell', processfile)
processcall = f"bash {processfilepath} {subject} {session} {studydir} -derivatives {derivatives} -dPar {dPar} -p {protocol}"
perform_process(processcall)

## An entry will have been put into the SubjectTracker-File, but just make sure that we end up by sorting this file according to input-column
# Read the SubjectTracker-File
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)
# sort the rows according to 'input'
df = df.sort_values(by = 'participant_id')
# and write to subjecttrackertsv-file
df.to_csv(subjecttrackertsv, sep="\t", index=False)

os.chdir(currdir)
